<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/index_structs/struct_indices/SQLIndexDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text-to-SQL Guide (Query Engine + Retriever)

This is a basic guide to LlamaIndex's Text-to-SQL capabilities.
1. We first show how to perform text-to-SQL over a toy dataset: this will do "retrieval" (sql query over db) and "synthesis".

2. We finally show you how to define a text-to-SQL retriever on its own.

**NOTE:** Any Text-to-SQL application should be aware that executing
arbitrary SQL queries can be a security risk. It is recommended to
take precautions as needed, such as using restricted roles, read-only
databases, sandboxing, etc.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [1]:
%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

INFO: pip is looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.9 MB/s

In [2]:
from llama_index.core import (
    VectorStoreIndex
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
from IPython.display import Markdown, display

### Create Database Schema

We use `sqlalchemy`, a popular SQL database toolkit, to create an empty `city_stats` Table

In [4]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    Float
)

In [11]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float
import pandas as pd

# Step 1: Create an in-memory SQLite database
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# Step 2: Define the 'products' table
products_table = Table(
    'products',
    metadata_obj,
    Column('product', String),
    Column('category', String),
    Column('sub_category', String),
    Column('brand', String),
    Column('sale_price', Float),
    Column('market_price', Float),
    Column('type', String),
    Column('rating', Float),
    Column('description', String),
    Column('quantity', Integer)
)

# Step 3: Create the table in the database
metadata_obj.create_all(engine)

# Step 4: Read data from a CSV file into a DataFrame
# Assuming your CSV file is named 'products.csv'
df = pd.read_csv('df.csv')

# Step 5: Drop the 'index' column from the DataFrame if it exists
df = df.drop(columns=['index'], errors='ignore')

# Step 6: Insert the data into the 'products' table without including the DataFrame index
df.to_sql('products', con=engine, if_exists='append', index=False)

# Verify by querying the data (Optional)
with engine.connect() as conn:
    result = conn.execute(products_table.select())
    for row in result:
        print(row)

Streaming output truncated to the last 5000 lines.
('Gulab Jal - Premium Rose Water', 'Beauty & Hygiene', 'Health & Medicine', 'Sri Sri Tattva', 65.0, 65.0, 'Ayurveda', None, 'Feel fresh with every spray of Sri Sri Tattva Gulab Jal (Rose Water). Boosted with the richness of rose extract, Gulab Jal acts as a natural skin ton ... (322 characters truncated) ...  Gulab\xa0Jal is\xa0a very good toner for skin, which helps in tightening the skin.Refreshing: The goodness of rose helps in relaxing the tired skin.', 24)
('Alcohol Based Hand Sanitiser With Refreshing Lemon', 'Beauty & Hygiene', 'Bath & Hand Wash', 'Bodyguard', 200.0, 250.0, 'Hand Wash & Sanitizers', 4.1, 'BodyGuard Hand Sanitizer\r\nalcohol-based\r\nrefreshing lemon\r\nKills 99.9% of germs\r\n\r\nBodyGuard is a protection and wellness range from Sirona ... (451 characters truncated) ... dyGuard Hand Sanitizer on the go. Formulated with the goodness of lemon, it kills 99.9% of germs, helping you lower the risk of contracting dise

### Define SQL Database

We first define our `SQLDatabase` abstraction (a light wrapper around SQLAlchemy).

In [12]:
from llama_index.core import SQLDatabase
from google.colab import userdata
GROQ_API_KEY = "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"

In [13]:
sql_database = SQLDatabase(engine, include_tables=["products"])

We add some testing data to our SQL database.

### Query Index

We first show how we can execute a raw SQL query, which directly executes over the table.

In [14]:
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

In [15]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Part 1: Text-to-SQL Query Engine
Once we have constructed our SQL database, we can use the NLSQLTableQueryEngine to
construct natural language queries that are synthesized into SQL queries.

Note that we need to specify the tables we want to use with this query engine.
If we don't the query engine will pull all the schema context, which could
overflow the context window of the LLM.

In [20]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["products"], llm=llm, embed_model=embed_model
)
query_str = "Tell me about Nivea products"
response = query_engine.query(query_str)

In [21]:
display(Markdown(f"<b>{response}</b>"))

<b>Nivea offers a wide range of products in the beauty and hygiene category, with a total of 64 products in their lineup. These products have an average rating of 4.2 out of 5 stars, indicating high customer satisfaction.

The top-rated products from Nivea include the Sensitive Cooling Shaving Gel with a perfect 5-star rating, followed closely by the Whitening Talc Touch Deodorant and the Creme Care Soap for Hands and Body, both with a 4.7-star rating.

Nivea's product range covers a variety of categories, including soaps, body lotions, deodorants, face washes, and lip balms. They offer products suitable for different skin types, including sensitive skin, dry skin, and oily skin.

Some popular products from Nivea include the Soft Aloe Moisturising Cream, the Whitening Smooth Skin Women Deodorant, and the Milk Delights Face Wash with Turmeric for Acne Prone Skin.

Overall, Nivea's products are known for their high quality and effectiveness, making them a popular choice among consumers.</b>

This query engine should be used in any case where you can specify the tables you want
to query over beforehand, or the total size of all the table schema plus the rest of
the prompt fits your context window.

You can also add additional context information for each table schema you define.